In [11]:
import numpy as np
from scipy.special import expit
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import mean_squared_error
from models import GibbsSamplerLLFM


In [12]:
def generate_synthetic(T=150, S=3, K_true=2, seed=0):

    np.random.seed(seed)

    # ----- True latent features -----
    Z_true = np.zeros((T, K_true))
    Z_true[:75, 0] = 1
    Z_true[100:, 1] = 1

    # ----- True weights -----
    W_true = np.zeros((K_true, S))
    W_true[0,0] = 5
    W_true[0,1] = 5
    W_true[1,2] = 5


    # ----- True bias -----
    b_true = np.array([-3, -3, -3])

    # ----- Generate observations -----
    logits = Z_true @ W_true + b_true
    P_true = expit(logits)
    Y = np.random.binomial(1, P_true)

    return Y, Z_true, W_true, b_true, P_true

Y, Z_true, W_true, b_true, P_true = generate_synthetic()
print("Generated synthetic data Y with shape:", Y.shape)
print(Y)
print(P_true)


Generated synthetic data Y with shape: (150, 3)
[[1 1 0]
 [1 1 0]
 [1 0 1]
 [1 1 0]
 [1 0 0]
 [1 1 0]
 [1 1 1]
 [1 1 0]
 [1 1 0]
 [0 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 0 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 0 0]
 [0 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 0 0]
 [1 1 0]
 [1 0 0]
 [1 1 0]
 [1 1 0]
 [1 0 0]
 [1 1 1]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [0 1 1]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 1]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [0 1 0]
 [1 1 0]
 [1 0 0]
 [1 1 0]
 [1 1 0]
 [1 0 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [0 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 1 0]
 [1 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 1 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 1 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1

In [13]:
# ---- Instantiate your sampler ----
sampler = GibbsSamplerLLFM(
    Data=Y,
    K=10,              
    alpha=0.5,
    sigma_w=2.0,
    sigma_b=0.5,
    mu_b=-5.0,
    n_iter=5000,
    burn=1000,
    n_subsample=1000
)



In [14]:

# ---- Run MCMC ----
sampler.run()

Average number of active features across iterations: 3.37


In [15]:
# ---- Posterior means ----
W_post, b_post, Z_post = sampler.get_posterior_samples()
W_post = np.array(W_post)
b_post = np.array(b_post)
Z_post = np.array(Z_post)

feature_counts = np.array([Z.sum(axis=0) for Z in Z_post])




In [17]:
import numpy as np

# ---- Get posterior samples ----
W_post, b_post, Z_post = sampler.get_posterior_samples()
Z_post = np.array(Z_post)  # shape: (n_samples, T, K)
W_post = np.array(W_post)  # shape: (n_samples, K, S)
b_post = np.array(b_post)  # shape: (n_samples, S)

n_samples, T, K = Z_post.shape
S = W_post.shape[2]

threshold = 12

# Normal dictionary
groups = {}
zeros = 0
for it in range(n_samples):
    Z = Z_post[it]   # (T, K)
    W = W_post[it]   # (K, S)
    b = b_post[it]

    usage = Z.sum(axis=0)

    # Identify active features
    active_idx = np.where(usage > threshold)[0]
    k_active = len(active_idx)

    if k_active == 0:

        zeros += 1
        continue  # Skip samples with no active features

    # Sort active features by usage descending
    sorted_idx = active_idx[np.argsort(usage[active_idx])[::-1]]

    W_perm = W[sorted_idx, :]

    # ---- Manually create group if needed ----
    if k_active not in groups:
        groups[k_active] = []

    groups[k_active].append((W_perm, b))


# ---- Report results ----
print("Posterior grouping by number of active features\n")
print(f"Number of samples with zero active features: {zeros}\n")
for k in sorted(groups.keys()):
    samples = groups[k]
    n_group = len(samples)

    print(f"Group with {k} active features:")
    print(f"  Number of posterior samples: {n_group}")

    W_stack = np.array([w for w, _ in samples])   # (n_group, k, S)
    b_stack = np.array([b for _, b in samples])   # (n_group, S)

    W_mean = W_stack.mean(axis=0)
    b_mean = b_stack.mean(axis=0)

    print("  Average weights:")
    print(W_mean)

    print("  Average bias:")
    print(b_mean)
    print("-" * 40)



Posterior grouping by number of active features

Number of samples with zero active features: 0

Group with 1 active features:
  Number of posterior samples: 89
  Average weights:
[[ 1.4680417   1.11306289 -8.56376648]]
  Average bias:
[-1.05245868 -1.03822733 -0.48463459]
----------------------------------------
Group with 2 active features:
  Number of posterior samples: 569
  Average weights:
[[ 1.72470286  1.32898924 -9.73405149]
 [-6.36253243 -6.61372742  1.52567962]]
  Average bias:
[-0.83414414 -0.88582812 -0.777894  ]
----------------------------------------
Group with 3 active features:
  Number of posterior samples: 278
  Average weights:
[[ 1.61164627  1.21580596 -8.28666287]
 [-3.66337654 -3.71033979 -0.26241187]
 [-2.80426502 -2.91136189 -0.10937644]]
  Average bias:
[-0.84488258 -0.83572993 -0.81853107]
----------------------------------------
Group with 4 active features:
  Number of posterior samples: 57
  Average weights:
[[ 1.33724584  0.93705926 -6.61300986]
 [-2.831

In [7]:

import numpy as np
from scipy.special import expit

# Convert lists of posterior samples to arrays
# samples_Z: list of (T, K), samples_W: list of (K, S), samples_b: list of (S,)
Z_array = np.array(Z_post)  # shape: (n_samples, T, K)
W_array = np.array(W_post)  # shape: (n_samples, K, S)
b_array = np.array(b_post)  # shape: (n_samples, S)

# Compute linear predictor for each posterior sample
# Shape: (n_samples, T, S)
eta_array = np.einsum('nTk,nKs->nTs', Z_array, W_array) + b_array[:, None, :]

# Apply sigmoid to get probabilities
P_post_array = expit(eta_array)

# Average over posterior samples to get posterior mean probability
P_post_mean = np.mean(P_post_array, axis=0)  # shape: (T, S)

print("Posterior mean probabilities shape:", P_post_mean.shape)
# P_post_mean[t, s] is the posterior mean probability of y[t, s] = 1

Posterior mean probabilities shape: (200, 4)


In [8]:

print(f'first 5: {P_true[:5]}')
print(f'last 5: {P_true[-5:]}')
print(f'P_post_mean first 5: {P_post_mean[:5]}')
print(f'P_post_mean last 5: {P_post_mean[-5:]}')

first 5: [[0.88079708 0.04742587 0.04742587 0.88079708]
 [0.88079708 0.04742587 0.04742587 0.88079708]
 [0.88079708 0.04742587 0.04742587 0.88079708]
 [0.88079708 0.04742587 0.04742587 0.88079708]
 [0.88079708 0.04742587 0.04742587 0.88079708]]
last 5: [[0.88079708 0.88079708 0.04742587 0.04742587]
 [0.88079708 0.88079708 0.04742587 0.04742587]
 [0.88079708 0.88079708 0.04742587 0.04742587]
 [0.88079708 0.88079708 0.04742587 0.04742587]
 [0.88079708 0.88079708 0.04742587 0.04742587]]
P_post_mean first 5: [[0.50503783 0.12527392 0.06526823 0.37663449]
 [0.46225779 0.16668754 0.09222597 0.35412327]
 [0.46349478 0.17512705 0.10080571 0.36984361]
 [0.50250758 0.12687998 0.06586019 0.36649709]
 [0.50178207 0.12313375 0.06569088 0.38537296]]
P_post_mean last 5: [[0.49057412 0.16951726 0.08479603 0.33976924]
 [0.48832985 0.17518506 0.08605647 0.33762389]
 [0.48817762 0.17487245 0.08951938 0.33856437]
 [0.41546831 0.22170545 0.13212784 0.34772555]
 [0.49502107 0.1704351  0.08669194 0.338816  ]